# Notes for Processing the Ashtabula and Erie county precinct data sets.

Using counts by matching candidate names.  Need to confirm with ?
Erie - 62 precincts
Ashtabula - 127 precincts

I tested the regex in Sublime Text first, then attempted to reproduce results in the shell.

## Regex patterns for matching candidates:

* Obama/Biden line: `^.*Obama.*$`
* McCain/Palin line: `^.*McCain.*$`

## For matching precinct information:

* Erie file precinct name:  `\d{4}\s[A-Z]{2,3}.*$`
Returns something of the form 0032 MIL VILL.  The 2,3 is because some abbreviations are only 2 characters, comma works as an OR.
* Ashtabula file precinct name: `\d{4}\s\d{2,3}.*$`

## From the command line: grep or ack

* grep "Biden" erie-pa.txt
    - with c gives count of 62
* grep grep `"B.*\sO.*$"` erie-pa.txt, matches whole line
    - with gives count of 62
    
I couldn't get grep or egrep to match the \d or [:digits:] patterns from the command line.

ack was more helpful

* ack "\d{4}\s\d{2,3}.*$" ashtabula-oh.txt

I was able to get egrep to work using the Erie file pattern on 12/07/16.



In [2]:
!egrep "Biden" data/erie-pa/erie-pa.txt

           Barack Obama/Joe Biden (DEM) .  .  .  .  .    388   45.92
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    382   57.62
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    291   45.61
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    273   49.01
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    358   44.97
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    405   56.88
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    139   36.48
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    630   72.50
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    555   80.67
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    425   62.04
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    624   74.02
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    511   65.01
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    608   71.36
           Barack Obama/Joe Biden (DEM) .  .  .  .  .    549   69.76
           Barack Obama/Joe Biden 

In [6]:
%bash
egrep '\d{4}\s\d{2,3}.*$' data/ohio-ashtabula/ashtabula-oh.txt

SyntaxError: invalid syntax (<ipython-input-6-1da4e4bbcc44>, line 2)

# Working with the Grep data

One solution is to use cut from the command line to pull out the characters at the end of the line.  I couldn't find a convenient switch to use cut to remove the characters at end of line so I used rev.

So this is the data from Erie county, in the same order as the original file with votes in the first column and percentage in the second.

A similar process can be used for data from both parties.

So the final output desired is for a csv file with the precinct name, votes for party1, percent for party1, votes for party2, percent for party2.



In [11]:
!rev data/dem.txt | cut -c1-11 | rev | tr -s ' ' ',' # gives me a comma separated list

388,45.92
382,57.62
291,45.61
273,49.01
358,44.97
405,56.88
139,36.48
630,72.50
555,80.67
425,62.04
624,74.02
511,65.01
608,71.36
549,69.76
616,69.14
554,74.97
616,70.08
448,73.08
460,74.43
551,77.06
473,65.69
624,63.09
244,47.38
248,58.35
347,55.97
307,55.62
287,54.98
211,53.55
121,41.58
203,57.18
241,50.74
222,41.42
319,50.63
419,46.92
302,48.24
414,50.49
354,46.40
330,40.79
315,46.19
309,44.21
351,48.82
293,57.91
427,58.17
275,55.78
242,44.40
342,38.30
365,52.52
513,61.44
484,48.30
307,42.34
355,58.39
112,49.78
405,53.36
360,52.33
449,55.23
345,54.68
433,48.49
409,53.19
295,54.53
264,52.28
223,51.74
226,45.93


In [12]:
!paste -d ',' data/erie-prec.txt data/dem-votes-erie.txt data/rep-votes-erie.txt | sed 's/^[ \t]*//;s/[ \t]*$//'

# the final sed command is to strip leading and trailing whitespace

0001 HUR A,388,45.92,446,52.78
0002 HUR B,382,57.62,274,41.33
0003 HUR C,291,45.61,338,52.98
0004 HUR D,273,49.01,272,48.83
0005 HUR E,358,44.97,432,54.27
0006 HUR F,405,56.88,286,40.17
0007 SAN A,139,36.48,235,61.68
0008 SAN B,630,72.50,227,26.12
0009 SAN C,555,80.67,126,18.31
0010 SAN D,425,62.04,246,35.91
0011 SAN E,624,74.02,205,24.32
0012 SAN F,511,65.01,267,33.97
0013 SAN G,608,71.36,230,27.00
0014 SAN H,549,69.76,223,28.34
0015 SAN I,616,69.14,261,29.29
0016 SAN J,554,74.97,173,23.41
0017 SAN K,616,70.08,241,27.42
0018 SAN L,448,73.08,154,25.12
0019 SAN M,460,74.43,140,22.65
0020 SAN N,551,77.06,153,21.40
0021 SAN O,473,65.69,218,30.28
0022 SAN P,624,63.09,344,34.78
0023 VER 1-A,244,47.38,261,50.68
0024 VER 2-A,248,58.35,164,38.59
0025 VER 2-B,347,55.97,259,41.77
0026 VER 3-A,307,55.62,240,43.48
0027 VER 3-B,287,54.98,219,41.95
0028 BER VILL,211,53.55,175,44.42
0029 KI VILL,121,41.58,167,57.39
0030 BV VILL,203,57.18,147,41.41
0031 CAST VILL,241,50.74,222,46.74
0032 MIL VILL,222,

# for the ashtabula data
# grep expressions
grep "Obama" ashtabula-oh.txt > dem.txt                                                                               
grep "McCain" ashtabula-oh.txt > rep.txt

# uses a sed expression to delete extra leading comma

!rev rep.txt | cut -c1-11 | rev | tr -s ' ' ',' | sed 's/^,//g' > rep-votes-ash.txt

# for the dems
!rev dem.txt | cut -c1-11 | rev | tr -s ' ' ',' > dem-votes-ash.txt

ack "\d{4}\s\d{2,3}.*$" ashtabula-oh.txt | sed 's/^[ \t]*//;s/[ \t]*$//' > ashtabula-prec.txt

paste -d ',' ashtabula-prec.txt dem-votes-ash.txt rep-votes-ash.txt | sed 's/^[ \t]*//;s/[ \t]*$//' > ash-out.csv